In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, math, time
import pickle
from sklearn import model_selection, datasets, linear_model, metrics, svm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

### Загрузим файл и разобъем на train/test:

In [13]:
data = pd.read_csv('data/' + 'calibration_dots3.csv')

full_train_data = data.iloc[0:36, :]
full_test_data = data.iloc[36:, :]

print('Shapes of train:{}   test:{}   and all:{}'.format(full_train_data.shape, full_test_data.shape, data.shape))

Shapes of train:(36, 9)   test:(3, 9)   and all:(39, 9)


## Точки калибровки
### Представлены в формате (координаты с 1cam), (координаты с 2cam), (реальные координаты)

In [3]:
data.head(3)

,x,y,r,x2,y2,r2,x_real,y_real,z_real
0,212,344,15,347,326,7,-15,-15,0
1,209,237,14,357,339,8,0,-15,0
2,214,130,14,361,359,9,15,-15,0


### Первая модель находит координату z_real как среднее выходов двух моделей по y, y2 и по x, x2

In [56]:
features = ['y', 'x2']
train_labels = full_train_data[['z_real']]
test_labels = full_test_data[['z_real']]
train_data = full_train_data[features]
test_data = full_test_data[features]

poly1 = PolynomialFeatures(2)
scaler1 = StandardScaler()
regressor1 = linear_model.LinearRegression()

pipeline1 = Pipeline(steps = [('polynomial', poly1), ('scaling', scaler1), ('regression', regressor1)])
pipeline1.fit(train_data, train_labels)

pred = pipeline1.predict(test_data)
mae = metrics.mean_absolute_error(test_labels, pred)
print('MAE = ',  mae)
print('Coefficients: ', *regressor1.coef_)

MAE =  0.31933886506035886
Coefficients:  [ 0.         -1.59965462  3.9308887  -2.40994563 -9.3820763   2.0147859 ]


In [54]:
features = ['x', 'y2']
train_labels = full_train_data[['z_real']]
test_labels = full_test_data[['z_real']]
train_data = full_train_data[features]
test_data = full_test_data[features]

poly2 = PolynomialFeatures(2)
scaler2 = StandardScaler()
regressor2 = linear_model.LinearRegression()

pipeline2 = Pipeline(steps = [('polynomial', poly2), ('scaling', scaler2), ('regression', regressor2)])
pipeline2.fit(train_data, train_labels)

pred = pipeline2.predict(test_data)
mae = metrics.mean_absolute_error(test_labels, pred)
print('MAE = ',  mae)
print('Coefficients: ', *regressor2.coef_)

MAE =  0.4303866491587443
Coefficients:  [  0.          -4.49916939 -13.54758999   0.16032307   8.35585825
  -3.74861091]


### Вторая модель находит координаты x_real, y_real по x, x2, z_real

In [55]:
features = ['x', 'x2', 'z_real']
train_labels = full_train_data[['x_real', 'y_real']]
test_labels = full_test_data[['x_real', 'y_real']]
train_data = full_train_data[features]
test_data = full_test_data[features]

poly3 = PolynomialFeatures(2)
scaler3 = StandardScaler()
regressor3 = MultiOutputRegressor(linear_model.LinearRegression())

pipeline3 = Pipeline(steps = [('polynomial', poly3), ('scaling', scaler3), ('regression', regressor3)])
pipeline3.fit(train_data, train_labels)

pred = pipeline3.predict(test_data)
mae = metrics.mean_absolute_error(test_labels, pred, multioutput='raw_values')
print('MAE = ',  *mae)

print('\nCoefficients of x_real part: ', regressor3.estimators_[0].coef_, '\n')
print('Coefficients of y_real part: ', regressor3.estimators_[1].coef_, '\n')

MAE =  0.19551530139657705 0.1506113135961822

Coefficients of x_real part:  [ 0.          2.56862393 -7.96972716 -1.07833142  1.26938974 11.18486378
  1.12948224  0.25593143 -0.3219025   0.40289879] 

Coefficients of y_real part:  [  0.          -7.95969178 -22.02553762   1.53436652   0.17625752
   9.4188145   -0.79749844   2.41683537  -0.38577829  -0.36742528] 



## Сохраним все 3 модели в один файл

In [60]:
models = [pipeline1, pipeline2, pipeline3]

# pickle.dump(pipeline1, open('data/calibration_model_1_1.sav', 'wb')) # save model

with open('data/calibration_models.sav', "wb") as file:
    for model in models:
         pickle.dump(model, file)

#### Проверяем:

In [65]:
my_models = []
with open('data/calibration_models.sav', "rb") as file:
    while True:
        try:
            my_models.append(pickle.load(file))
        except EOFError:
            break

pred1 = my_models[0].predict(full_test_data[['y', 'x2']])
pred2 = my_models[1].predict(full_test_data[['x', 'y2']])
print(pred2)
avg = []
for i in range(len(pred1)):
    avg.append(float(pred1[i]+pred2[i])/2)
print(avg)
mae = metrics.mean_absolute_error(test_labels, avg, multioutput='raw_values')
print('MAE = ',  *mae)

[[ 0.45325717]
 [20.58668084]
 [29.74877806]]
[0.39906273189801844, 20.519244091195404, 29.79371855176477]
MAE =  0.374862757109551
